In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
sns.set()

In [2]:
filename = "../../results_aggr/app_bootstrap_vm.csv"
outfilePrefix = "../../results_aggr/app_bootstrap_vm_plots"
df_values = pd.read_csv(filename, names=["commit", "type", "CI_min", "Median", "CI_max", "jump", "trend"], skiprows=1)
df_values.sort_values(by=["commit"], inplace=True)

In [3]:
for type in df_values.type.unique():
    print(f"Plotting type {type}")

    measurements = df_values.loc[(df_values['type'].str.startswith(type, na=False))]

    #Find % set y-limits
    min = -5
    max = 5
    for minSmall in measurements.CI_min.unique():
        if minSmall < min:
            min = minSmall

    for maxSmall in measurements.CI_max.unique():
        if maxSmall > max:
            max = maxSmall


    sns.set(style="whitegrid")
    plot, axes = plt.subplots(figsize=(15,9))
    axes.set_ylim([min, max])
    axes.set_xlim([0, 70])
    axes.set_xticks([0,10,20,30,40,50,60,70])

    uptrendMarker="$↗$"
    downtrendMarker="$↘$"
    jumpSize=1000
    trendSize=700
    fillColor="#bac7e6"
    edgeColor='black'

    #Plot CIs
    plot = sns.lineplot(data=measurements, x="commit", y="CI_min", color="royalblue", zorder=2)
    plot = sns.lineplot(data=measurements, x="commit", y="CI_max", color="royalblue", zorder=2)
    x_commits = measurements.commit
    plot.fill_between(x=x_commits, y1=measurements.CI_min, y2=measurements.CI_max, facecolor =fillColor, zorder=1)

    #Plot Median line
    plot = sns.lineplot(data=measurements, x="commit", y="Median", color="black")

    #PLot Up&Down Changes
    ups = measurements.loc[(measurements['jump'].astype(str).str.startswith("potential up", na=False))]
    plot = sns.regplot(data=ups,x="commit", y="Median", marker="^", fit_reg=False, scatter_kws={"color":"orange","alpha":1,"s":jumpSize, "zorder":3, 'edgecolor':edgeColor})
    ups = measurements.loc[(measurements['jump'].astype(str).str.startswith("definite up", na=False))]
    plot = sns.regplot(data=ups,x="commit", y="Median", marker="^", fit_reg=False, scatter_kws={"color":"red","alpha":1,"s":jumpSize, "zorder":3, 'edgecolor':edgeColor})

    ups = measurements.loc[(measurements['trend'].astype(str).str.startswith("potential up", na=False))]
    plot = sns.regplot(data=ups,x="commit", y="Median", marker=uptrendMarker, fit_reg=False, scatter_kws={"color":"orange","alpha":1,"s":trendSize, "zorder":3, 'edgecolor':edgeColor})
    ups = measurements.loc[(measurements['trend'].astype(str).str.startswith("definite up", na=False))]
    plot = sns.regplot(data=ups,x="commit", y="Median", marker=uptrendMarker, fit_reg=False, scatter_kws={"color":"red","alpha":1,"s":trendSize, "zorder":3, 'edgecolor':edgeColor})

    downs = measurements.loc[(measurements['jump'].astype(str).str.startswith("potential down", na=False))]
    plot = sns.regplot(data=downs,x="commit", y="Median", marker="v", fit_reg=False, scatter_kws={"color":"orange","alpha":1,"s":jumpSize, "zorder":3, 'edgecolor':edgeColor})
    downs = measurements.loc[(measurements['jump'].astype(str).str.startswith("definite down", na=False))]
    plot = sns.regplot(data=downs,x="commit", y="Median", marker="v", fit_reg=False, scatter_kws={"color":"red","alpha":1,"s":jumpSize, "zorder":3, 'edgecolor':edgeColor})

    downs = measurements.loc[(measurements['trend'].astype(str).str.startswith("potential down", na=False))]
    plot = sns.regplot(data=downs,x="commit", y="Median", marker=downtrendMarker, fit_reg=False, scatter_kws={"color":"orange","alpha":1,"s":trendSize, "zorder":3, 'edgecolor':edgeColor})
    downs = measurements.loc[(measurements['trend'].astype(str).str.startswith("definite down", na=False))]
    plot = sns.regplot(data=downs,x="commit", y="Median", marker=downtrendMarker, fit_reg=False, scatter_kws={"color":"red","alpha":1,"s":trendSize, "zorder":3, 'edgecolor':edgeColor})

    legend_elements = [
        Line2D([0], [0], color='black', lw=1, label='Median Performance Change'),
        mpatches.Patch(color=fillColor, label='99% Confidence Interval'),
        Line2D([0], [0], marker='^', color='w', label='Potential Performance Jump (Up)', markerfacecolor='orange', markeredgecolor=edgeColor, markersize=16),
        Line2D([0], [0], marker='v', color='w', label='Definite Performance Jump (Down)', markerfacecolor='red', markeredgecolor=edgeColor, markersize=16),
        Line2D([0], [0], marker=uptrendMarker, color='w', label='Potential Performance Trend (Up)', markerfacecolor='orange', markeredgecolor=edgeColor, markersize=16),
        Line2D([0], [0], marker=downtrendMarker, color='w', label='Definite Performance Trend (Down)', markerfacecolor='red', markeredgecolor=edgeColor, markersize=16)
    ]

    if (type == "group-by queries"):
        plot.legend(fontsize="medium", loc=1, title='', handles=legend_elements, labelspacing=0.8)


    plot.set(ylim=(min, max))
    plt.savefig(f'{outfilePrefix}/{type}.png')
    plt.clf()


Plotting type inserts
Plotting type simple queries
Plotting type group-by queries


<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>

<Figure size 1080x648 with 0 Axes>